Aqui ocorrem:
1. a detecção e a substituição de outliers pelo limite inferior ou superior em variáveis quantitativas;
2. outra execução do "VarianceThreshold"com threshold = 0;
3. a padronização de variáveis quantitativas com "StandardScaler";
4. a remoção de variáveis com correlação maior do que 0.8, a partir da geração da matriz de correlação com o método ".corr" do pd.DataFrame;
5. o one hot encoding das variáveis qualitativas;
6. a alteração da variável "no_curso" para incluir município e grau acadêmico do curso (o nome do curso não é utilizado no agrupamento, nem o seu código);
6. a remoção da variável que indica se o curso está situado na capital;

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder

C:\Users\ebony\AppData\Local\Temp\ipykernel_19856\3986118292.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('cursos-ufrpe-pos-selecao-inicial.csv',encoding='ISO-8859-1',sep=';')

In [4]:
df

,in_capital,co_curso,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_nova,qt_vg_remanesc,qt_inscrito_total,qt_inscrito_total_diurno,qt_inscrito_total_noturno,qt_insc_vg_nova,qt_insc_vg_remanesc,qt_ing,qt_ing_fem,qt_ing_masc,qt_ing_diurno,qt_ing_noturno,qt_ing_vg_nova,qt_ing_vestibular,qt_ing_enem,qt_ing_vg_remanesc,qt_ing_outra_forma,qt_ing_0_17,qt_ing_18_24,qt_ing_25_29,qt_ing_30_34,qt_ing_35_39,qt_ing_40_49,qt_ing_50_59,qt_ing_60_mais,qt_ing_branca,qt_ing_preta,qt_ing_parda,qt_ing_amarela,qt_ing_indigena,qt_ing_cornd,qt_mat,qt_mat_fem,qt_mat_masc,qt_mat_diurno,qt_mat_noturno,qt_mat_0_17,qt_mat_18_24,qt_mat_25_29,qt_mat_30_34,qt_mat_35_39,qt_mat_40_49,qt_mat_50_59,qt_mat_60_mais,qt_mat_branca,qt_mat_preta,qt_mat_parda,qt_mat_amarela,qt_mat_indigena,qt_mat_cornd,qt_conc,qt_conc_fem,qt_conc_masc,qt_conc_diurno,qt_conc_noturno,qt_conc_18_24,qt_conc_25_29,qt_conc_30_34,qt_conc_35_39,qt_conc_40_49,qt_conc_50_59,qt_conc_60_mais,qt_conc_branca,qt_conc_preta,qt_conc_parda,qt_conc_amarela,qt_conc_indigena,qt_conc_cornd,qt_ing_nacbras,qt_mat_nacbras,qt_mat_nacestrang,qt_conc_nacbras,qt_aluno_deficiente,qt_ing_deficiente,qt_mat_deficiente,qt_conc_deficiente,qt_ing_reserva_vaga,qt_ing_rvredepublica,qt_ing_rvetnico,qt_ing_rvsocial_rf,qt_ing_rvoutros,qt_mat_reserva_vaga,qt_mat_rvredepublica,qt_mat_rvetnico,qt_mat_rvsocial_rf,qt_mat_rvoutros,qt_conc_reserva_vaga,qt_conc_rvredepublica,qt_conc_rvetnico,qt_conc_rvsocial_rf,qt_conc_rvoutros,qt_sit_trancada,qt_sit_desvinculado,qt_ing_procescpublica,qt_ing_procescprivada,qt_mat_procescpublica,qt_mat_procescprivada,qt_mat_procnaoinformada,qt_conc_procescpublica,qt_conc_procescprivada,qt_conc_procnaoinformada,qt_apoio_social,qt_ing_apoio_social,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica,no_curso,no_municipio,no_cine_area_geral,tp_grau_academico,enade_faixa,cpc_faixa
0,0.0,1516690.0,80.0,80.0,0.0,80.0,0.0,360.0,360.0,0.0,360.0,0.0,76.0,14.0,62.0,76.0,0.0,76.0,4.0,72.0,0.0,0.0,2.0,72.0,0.0,1.0,0.0,1.0,0.0,0.0,27.0,5.0,43.0,1.0,0.0,0.0,123.0,19.0,104.0,123.0,0.0,2.0,116.0,2.0,2.0,0.0,1.0,0.0,0.0,49.0,6.0,65.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76.0,123.0,0.0,0.0,2.0,2.0,2.0,0.0,63.0,37.0,20.0,17.0,26.0,63.0,37.0,20.0,17.0,26.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,65.0,11.0,89.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,engenharia_de_computacao,belo_jardim,"engenharia,_producao_e_construcao",bacharelado,SC,SC
1,0.0,1516692.0,80.0,80.0,0.0,80.0,0.0,282.0,282.0,0.0,282.0,0.0,65.0,17.0,48.0,65.0,0.0,65.0,12.0,53.0,0.0,0.0,3.0,53.0,4.0,2.0,2.0,1.0,0.0,0.0,25.0,4.0,31.0,0.0,0.0,5.0,104.0,25.0,79.0,104.0,0.0,2.0,86.0,8.0,4.0,2.0,2.0,0.0,0.0,37.0,9.0,52.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,19.0,14.0,9.0,24.0,42.0,20.0,15.0,10.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,44.0,21.0,63.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,engenharia_de_controle_e_automacao,belo_jardim,"engenharia,_producao_e_construcao",bacharelado,SC,SC
2,0.0,1516689.0,80.0,80.0,0.0,80.0,0.0,305.0,305.0,0.0,305.0,0.0,37.0,13.0,24.0,37.0,0.0,37.0,4.0,33.0,0.0,0.0,2.0,24.0,6.0,2.0,1.0,2.0,0.0,0.0,11.0,5.0,18.0,2.0,0.0,1.0,72.0,29.0,43.0,72.0,0.0,2.0,53.0,8.0,4.0,2.0,3.0,0.0,0.0,25.0,7.0,35.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,18.0,11.0,8.0,13.0,30.0,19.0,11.0,9.0,12.0,0.0,0.0,0.0,0.0,0.0,2.0,13.0,30.0,7.0,51.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,engenharia_hidrica,belo_jardim,"engenharia,_producao_e_construcao",bacharelado,SC,SC
3,0.0,1516693.0,80.0,80.0,0.0,80.0,0.0,301.0,301.0,0.0,301.0,0.0,54.0,30.0,24.0,54.0,0.0,54.0,11.0,43.0,0.0,0.0,5.0,44.0,4.0,1.0,0.0,0.0,0.0,0.0,13.0,3.0,35.0,1.0

In [5]:
df.shape

(46, 123)

In [6]:
def detect_outliers_iqr(data, threshold=1.5):
    """
    Detecta outliers em uma série de dados usando o método do intervalo interquartil (IQR).
    
    Parâmetros:
        data (pd.Series): A série de dados para verificar outliers.
        threshold (float): O fator multiplicativo do IQR usado para definir o limite para outliers.
        
    Retorna:
        outliers (int): O número de amostras com outliers.
    """
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    return len(outliers)

In [7]:
# Calcula a quantidade de amostras com outliers para cada coluna numérica
outliers_count = {}
for column in df.select_dtypes(include=['number']):
    outliers_count[column] = detect_outliers_iqr(df[column])

# Exibe a quantidade de amostras com outliers por coluna
for column, count in outliers_count.items():
    print(f"Coluna '{column}': {count} amostras com outliers")

Coluna 'in_capital': 0 amostras com outliers
Coluna 'co_curso': 0 amostras com outliers
Coluna 'qt_vg_total': 9 amostras com outliers
Coluna 'qt_vg_total_diurno': 0 amostras com outliers
Coluna 'qt_vg_total_noturno': 0 amostras com outliers
Coluna 'qt_vg_nova': 15 amostras com outliers
Coluna 'qt_vg_remanesc': 0 amostras com outliers
Coluna 'qt_inscrito_total': 4 amostras com outliers
Coluna 'qt_inscrito_total_diurno': 3 amostras com outliers
Coluna 'qt_inscrito_total_noturno': 1 amostras com outliers
Coluna 'qt_insc_vg_nova': 4 amostras com outliers
Coluna 'qt_insc_vg_remanesc': 3 amostras com outliers
Coluna 'qt_ing': 4 amostras com outliers
Coluna 'qt_ing_fem': 1 amostras com outliers
Coluna 'qt_ing_masc': 0 amostras com outliers
Coluna 'qt_ing_diurno': 0 amostras com outliers
Coluna 'qt_ing_noturno': 0 amostras com outliers
Coluna 'qt_ing_vg_nova': 10 amostras com outliers
Coluna 'qt_ing_vestibular': 11 amostras com outliers
Coluna 'qt_ing_enem': 8 amostras com outliers
Coluna 'qt_

In [8]:
def replace_outliers_iqr(data, threshold=1.5):
    """
    Substitui outliers em uma série de dados pelos limites inferior ou superior do IQR.
    
    Parâmetros:
        data (pd.Series): A série de dados para substituir outliers.
        threshold (float): O fator multiplicativo do IQR usado para definir o limite para outliers.
        
    Retorna:
        data_with_replaced_outliers (pd.Series): A série de dados com outliers substituídos.
    """
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    
    data_with_replaced_outliers = data.copy()
    data_with_replaced_outliers[data_with_replaced_outliers < lower_bound] = lower_bound
    data_with_replaced_outliers[data_with_replaced_outliers > upper_bound] = upper_bound
    
    return data_with_replaced_outliers

In [9]:
# Substitui os outliers por limites do IQR para cada coluna numérica
for column in df.select_dtypes(include=['number']):
    df[column] = replace_outliers_iqr(df[column])

In [10]:
# Calcula a quantidade de amostras com outliers para cada coluna numérica
outliers_count = {}
for column in df.select_dtypes(include=['number']):
    outliers_count[column] = detect_outliers_iqr(df[column])

# Exibe a quantidade de amostras com outliers por coluna
for column, count in outliers_count.items():
    print(f"Coluna '{column}': {count} amostras com outliers")

Coluna 'in_capital': 0 amostras com outliers
Coluna 'co_curso': 0 amostras com outliers
Coluna 'qt_vg_total': 0 amostras com outliers
Coluna 'qt_vg_total_diurno': 0 amostras com outliers
Coluna 'qt_vg_total_noturno': 0 amostras com outliers
Coluna 'qt_vg_nova': 0 amostras com outliers
Coluna 'qt_vg_remanesc': 0 amostras com outliers
Coluna 'qt_inscrito_total': 0 amostras com outliers
Coluna 'qt_inscrito_total_diurno': 0 amostras com outliers
Coluna 'qt_inscrito_total_noturno': 0 amostras com outliers
Coluna 'qt_insc_vg_nova': 0 amostras com outliers
Coluna 'qt_insc_vg_remanesc': 0 amostras com outliers
Coluna 'qt_ing': 0 amostras com outliers
Coluna 'qt_ing_fem': 0 amostras com outliers
Coluna 'qt_ing_masc': 0 amostras com outliers
Coluna 'qt_ing_diurno': 0 amostras com outliers
Coluna 'qt_ing_noturno': 0 amostras com outliers
Coluna 'qt_ing_vg_nova': 0 amostras com outliers
Coluna 'qt_ing_vestibular': 0 amostras com outliers
Coluna 'qt_ing_enem': 0 amostras com outliers
Coluna 'qt_ing

In [11]:
df

,in_capital,co_curso,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_nova,qt_vg_remanesc,qt_inscrito_total,qt_inscrito_total_diurno,qt_inscrito_total_noturno,qt_insc_vg_nova,qt_insc_vg_remanesc,qt_ing,qt_ing_fem,qt_ing_masc,qt_ing_diurno,qt_ing_noturno,qt_ing_vg_nova,qt_ing_vestibular,qt_ing_enem,qt_ing_vg_remanesc,qt_ing_outra_forma,qt_ing_0_17,qt_ing_18_24,qt_ing_25_29,qt_ing_30_34,qt_ing_35_39,qt_ing_40_49,qt_ing_50_59,qt_ing_60_mais,qt_ing_branca,qt_ing_preta,qt_ing_parda,qt_ing_amarela,qt_ing_indigena,qt_ing_cornd,qt_mat,qt_mat_fem,qt_mat_masc,qt_mat_diurno,qt_mat_noturno,qt_mat_0_17,qt_mat_18_24,qt_mat_25_29,qt_mat_30_34,qt_mat_35_39,qt_mat_40_49,qt_mat_50_59,qt_mat_60_mais,qt_mat_branca,qt_mat_preta,qt_mat_parda,qt_mat_amarela,qt_mat_indigena,qt_mat_cornd,qt_conc,qt_conc_fem,qt_conc_masc,qt_conc_diurno,qt_conc_noturno,qt_conc_18_24,qt_conc_25_29,qt_conc_30_34,qt_conc_35_39,qt_conc_40_49,qt_conc_50_59,qt_conc_60_mais,qt_conc_branca,qt_conc_preta,qt_conc_parda,qt_conc_amarela,qt_conc_indigena,qt_conc_cornd,qt_ing_nacbras,qt_mat_nacbras,qt_mat_nacestrang,qt_conc_nacbras,qt_aluno_deficiente,qt_ing_deficiente,qt_mat_deficiente,qt_conc_deficiente,qt_ing_reserva_vaga,qt_ing_rvredepublica,qt_ing_rvetnico,qt_ing_rvsocial_rf,qt_ing_rvoutros,qt_mat_reserva_vaga,qt_mat_rvredepublica,qt_mat_rvetnico,qt_mat_rvsocial_rf,qt_mat_rvoutros,qt_conc_reserva_vaga,qt_conc_rvredepublica,qt_conc_rvetnico,qt_conc_rvsocial_rf,qt_conc_rvoutros,qt_sit_trancada,qt_sit_desvinculado,qt_ing_procescpublica,qt_ing_procescprivada,qt_mat_procescpublica,qt_mat_procescprivada,qt_mat_procnaoinformada,qt_conc_procescpublica,qt_conc_procescprivada,qt_conc_procnaoinformada,qt_apoio_social,qt_ing_apoio_social,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_ing_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular,qt_mob_academica,qt_mat_mob_academica,qt_conc_mob_academica,no_curso,no_municipio,no_cine_area_geral,tp_grau_academico,enade_faixa,cpc_faixa
0,0.0,1516690.0,80.0,80.0,0.0,80.0,0.0,360.000,360.00,0.00,360.000,0.000,76.000,14.000,62.0,76.0,0.0,76.0,0.0,72.000,0.0,0.0,2.000,72.000,0.0,1.0,0.0,1.0,0.0,0.0,27.0,5.0,43.000,1.0,0.0,0.0,123.0,19.000,104.0,123.0,0.0,2.000,116.000,2.000,2.0,0.0,1.0,0.0,0.0,49.0,6.0,65.00,2.0,1.000,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76.000,123.0,0.0,0.0,2.0,2.0,2.000,0.0,17.0,12.875,10.375,10.0,14.0,36.5,22.375,18.875,17.00,19.5,0.0,0.0,0.0,0.0,0.0,1.0,10.0,34.375,11.0,62.0,34.25,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000,1.000,0.0,1.00,0.0,0.0,0.0,0.0,engenharia_de_computacao,belo_jardim,"engenharia,_producao_e_construcao",bacharelado,SC,SC
1,0.0,1516692.0,80.0,80.0,0.0,80.0,0.0,282.000,282.00,0.00,282.000,0.000,65.000,17.000,48.0,65.0,0.0,65.0,0.0,53.000,0.0,0.0,3.000,53.000,4.0,2.0,2.0,1.0,0.0,0.0,25.0,4.0,31.000,0.0,0.0,5.0,104.0,25.000,79.0,104.0,0.0,2.000,86.000,8.000,4.0,2.0,2.0,0.0,0.0,37.0,9.0,52.00,1.0,0.000,5.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65.000,104.0,0.0,0.0,0.0,0.0,0.000,0.0,17.0,12.875,10.375,9.0,14.0,36.5,20.000,15.000,10.00,19.5,0.0,0.0,0.0,0.0,0.0,0.0,30.0,34.375,21.0,62.0,41.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.0,0.0,engenharia_de_controle_e_automacao,belo_jardim,"engenharia,_producao_e_construcao",bacharelado,SC,SC
2,0.0,1516689.0,80.0,80.0,0.0,80.0,0.0,305.000,305.00,0.00,305.000,0.000,37.000,13.000,24.0,37.0,0.0,37.0,0.0,33.000,0.0,0.0,2.000,24.000,6.0,2.0,1.0,2.0,0.0,0.0,11.0,5.0,18.000,2.0,0.0,1.0,72.0,29.000,43.0,72.0,0.0,2.000,53.000,8.000,4.0,2.0,3.0,0.0,0.0,25.0,7.0,35.00,3.0,1.000,1.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.000,72.0,0.0,0.0,0.0,0.0,0.000,0.0,17.0,12.875,10.375,8.0,13.0,30.0,19.000,11.000,9.00,12.0,0.0,0.0,0.0,0.0,0.0,2.0,13.0,30.000,7.0,51.0,34.25,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.00,0.0,0.0,0.0,0.0,engenharia_hidrica,belo_jardim,"engenharia,_producao_e_construcao",bacharelado,S

In [12]:
df.shape

(46, 123)

In [13]:
df1 = df.drop(['no_curso','no_municipio','no_cine_area_geral','tp_grau_academico','enade_faixa', 'cpc_faixa'],axis=1)

In [14]:
sel = VarianceThreshold(threshold=(0))
df_selected = sel.fit_transform(df1)
cols_selected = df1.columns[sel.get_support(indices=True)]
df_selected = pd.DataFrame(df_selected, columns=cols_selected)

In [15]:
df_selected

,in_capital,co_curso,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_remanesc,qt_inscrito_total,qt_inscrito_total_diurno,qt_inscrito_total_noturno,qt_insc_vg_nova,qt_insc_vg_remanesc,qt_ing,qt_ing_fem,qt_ing_masc,qt_ing_diurno,qt_ing_noturno,qt_ing_vg_nova,qt_ing_enem,qt_ing_vg_remanesc,qt_ing_outra_forma,qt_ing_0_17,qt_ing_18_24,qt_ing_25_29,qt_ing_30_34,qt_ing_35_39,qt_ing_40_49,qt_ing_50_59,qt_ing_branca,qt_ing_preta,qt_ing_parda,qt_ing_amarela,qt_ing_cornd,qt_mat,qt_mat_fem,qt_mat_masc,qt_mat_diurno,qt_mat_noturno,qt_mat_0_17,qt_mat_18_24,qt_mat_25_29,qt_mat_30_34,qt_mat_35_39,qt_mat_40_49,qt_mat_50_59,qt_mat_60_mais,qt_mat_branca,qt_mat_preta,qt_mat_parda,qt_mat_amarela,qt_mat_indigena,qt_mat_cornd,qt_conc,qt_conc_fem,qt_conc_masc,qt_conc_diurno,qt_conc_noturno,qt_conc_18_24,qt_conc_25_29,qt_conc_30_34,qt_conc_branca,qt_conc_preta,qt_conc_parda,qt_ing_nacbras,qt_mat_nacbras,qt_conc_nacbras,qt_aluno_deficiente,qt_ing_deficiente,qt_mat_deficiente,qt_ing_reserva_vaga,qt_ing_rvredepublica,qt_ing_rvetnico,qt_ing_rvsocial_rf,qt_ing_rvoutros,qt_mat_reserva_vaga,qt_mat_rvredepublica,qt_mat_rvetnico,qt_mat_rvsocial_rf,qt_mat_rvoutros,qt_sit_trancada,qt_sit_desvinculado,qt_ing_procescpublica,qt_ing_procescprivada,qt_mat_procescpublica,qt_mat_procescprivada,qt_conc_procescpublica,qt_conc_procescprivada,qt_apoio_social,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular
0,0.0,1516690.0,80.0,80.0,0.0,0.0,360.000,360.00,0.00,360.000,0.000,76.000,14.000,62.0,76.0,0.0,76.0,72.000,0.0,0.0,2.000,72.000,0.0,1.0,0.0,1.0,0.0,27.0,5.0,43.000,1.0,0.0,123.0,19.000,104.0,123.0,0.0,2.000,116.000,2.000,2.0,0.0,1.0,0.0,0.0,49.0,6.0,65.00,2.0,1.000,0.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,76.000,123.0,0.0,2.0,2.0,2.000,17.0,12.875,10.375,10.0,14.0,36.5,22.375,18.875,17.00,19.5,1.0,10.0,34.375,11.0,62.0,34.25,0.0,0.00,0.0,0.0,0.000,1.000,1.00,0.0
1,0.0,1516692.0,80.0,80.0,0.0,0.0,282.000,282.00,0.00,282.000,0.000,65.000,17.000,48.0,65.0,0.0,65.0,53.000,0.0,0.0,3.000,53.000,4.0,2.0,2.0,1.0,0.0,25.0,4.0,31.000,0.0,5.0,104.0,25.000,79.0,104.0,0.0,2.000,86.000,8.000,4.0,2.0,2.0,0.0,0.0,37.0,9.0,52.00,1.0,0.000,5.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,65.000,104.0,0.0,0.0,0.0,0.000,17.0,12.875,10.375,9.0,14.0,36.5,20.000,15.000,10.00,19.5,0.0,30.0,34.375,21.0,62.0,41.00,0.0,0.00,0.0,0.0,0.000,0.000,0.00,0.0
2,0.0,1516689.0,80.0,80.0,0.0,0.0,305.000,305.00,0.00,305.000,0.000,37.000,13.000,24.0,37.0,0.0,37.0,33.000,0.0,0.0,2.000,24.000,6.0,2.0,1.0,2.0,0.0,11.0,5.0,18.000,2.0,1.0,72.0,29.000,43.0,72.0,0.0,2.000,53.000,8.000,4.0,2.0,3.0,0.0,0.0,25.0,7.0,35.00,3.0,1.000,1.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,37.000,72.0,0.0,0.0,0.0,0.000,17.0,12.875,10.375,8.0,13.0,30.0,19.000,11.000,9.00,12.0,2.0,13.0,30.000,7.0,51.0,34.25,0.0,0.00,0.0,0.0,0.000,0.000,0.00,0.0
3,0.0,1516693.0,80.0,80.0,0.0,0.0,301.000,301.00,0.00,301.000,0.000,54.000,30.000,24.0,54.0,0.0,54.0,43.000,0.0,0.0,5.000,44.000,4.0,1.0,0.0,0.0,0.0,13.0,3.0,35.000,1.0,2.0,99.0,53.000,46.0,99.0,0.0,5.000,81.000,8.000,2.0,1.0,1.0,1.0,0.0,30.0,6.0,59.00,2.0,0.000,2.0,0.0,0.0,0.000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,54.000,99.0,0.0,0.0,0.0,0.000,17.0,12.875,10.375,10.0,14.0,36.5,21.000,18.000,14.00,19.0,0.0,12.0,34.375,14.0,62.0,35.00,0.0,0.00,0.0,0.0,0.000,1.000,1.00,0.0
4,0.0,1270379.0,140.0,140.0,0.0,20.0,790.000,790.00,0.00,784.000,6.000,106.000,46.000,60.0,106.0,0.0,102.0,64.000,1.0,2.5,7.875,73.000,16.0,1.0,2.0,4.0,1.0,39.0,15.0,50.000,1.0,1.0,356.0,147.000,209.0,356.0,0.0,7.875,265.000,57.000,12.0,5.0,7.0,1.0,1.0,122.0,34.0,193.00,5.0,1.000,1.0,7.0,2.0,5.000,7.0,0.00,3.0,4.0,0.0,1.0,0.0,6.0,106.000,356.0,7.0,2.0,0.0,1.000,17.0,12.875,10.375,10.0,14.0,36.5,22.375,18.875,18.25,19.5,11.0,59.0,34.375,34.0,62.0,217.00,3.0,4.00,42.0,37.0,1.000,2.000,2.00,0.0
5,0.0,1270383.0,140.0,140.0,0.0,20.0,683.000,683.00,0.00,683.000,0.000,119.000,44.000,75.0,119.0,0.0,107.5,104.000,0.0,0.0,5.000,80.000,11.0,9

In [16]:
df_selected.shape

(46, 92)

In [17]:
scaler = StandardScaler()
standard_data = scaler.fit_transform(df_selected.drop('co_curso', axis=1))

In [18]:
df_processed = pd.DataFrame(standard_data, columns=df_selected.drop('co_curso', axis=1).columns)

In [19]:
df_processed

,in_capital,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_remanesc,qt_inscrito_total,qt_inscrito_total_diurno,qt_inscrito_total_noturno,qt_insc_vg_nova,qt_insc_vg_remanesc,qt_ing,qt_ing_fem,qt_ing_masc,qt_ing_diurno,qt_ing_noturno,qt_ing_vg_nova,qt_ing_enem,qt_ing_vg_remanesc,qt_ing_outra_forma,qt_ing_0_17,qt_ing_18_24,qt_ing_25_29,qt_ing_30_34,qt_ing_35_39,qt_ing_40_49,qt_ing_50_59,qt_ing_branca,qt_ing_preta,qt_ing_parda,qt_ing_amarela,qt_ing_cornd,qt_mat,qt_mat_fem,qt_mat_masc,qt_mat_diurno,qt_mat_noturno,qt_mat_0_17,qt_mat_18_24,qt_mat_25_29,qt_mat_30_34,qt_mat_35_39,qt_mat_40_49,qt_mat_50_59,qt_mat_60_mais,qt_mat_branca,qt_mat_preta,qt_mat_parda,qt_mat_amarela,qt_mat_indigena,qt_mat_cornd,qt_conc,qt_conc_fem,qt_conc_masc,qt_conc_diurno,qt_conc_noturno,qt_conc_18_24,qt_conc_25_29,qt_conc_30_34,qt_conc_branca,qt_conc_preta,qt_conc_parda,qt_ing_nacbras,qt_mat_nacbras,qt_conc_nacbras,qt_aluno_deficiente,qt_ing_deficiente,qt_mat_deficiente,qt_ing_reserva_vaga,qt_ing_rvredepublica,qt_ing_rvetnico,qt_ing_rvsocial_rf,qt_ing_rvoutros,qt_mat_reserva_vaga,qt_mat_rvredepublica,qt_mat_rvetnico,qt_mat_rvsocial_rf,qt_mat_rvoutros,qt_sit_trancada,qt_sit_desvinculado,qt_ing_procescpublica,qt_ing_procescprivada,qt_mat_procescpublica,qt_mat_procescprivada,qt_conc_procescpublica,qt_conc_procescprivada,qt_apoio_social,qt_mat_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,qt_mat_ativ_extracurricular,qt_conc_ativ_extracurricular
0,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-0.860750,-0.372702,-0.629966,-0.856954,-0.839852,0.023957,-0.949849,0.780188,0.590679,-0.663030,0.201027,0.111880,-0.748999,-0.697633,-0.088235,0.743235,-1.593351,-0.998333,-1.244078,-0.708742,-0.849875,0.004966,-0.784481,0.546219,0.296756,-0.898681,-1.160605,-1.291390,-0.546268,-0.461069,-0.660136,-0.098134,-0.749292,-1.631047,-1.349439,-1.114264,-0.996644,-1.021086,-0.750317,-0.866404,-1.370470,-1.244323,-0.565178,-0.101828,-1.302557,-1.103646,-0.950324,-1.080212,-0.860915,-0.613991,-0.980197,-1.062303,-0.824470,-0.921642,-0.896293,-1.058423,0.023957,-1.160393,-1.103646,-0.669994,1.103235,-0.542176,1.758524,1.856723,1.825778,1.951627,1.909683,1.885189,1.925977,2.045059,2.002720,1.912801,-0.921816,-1.580111,1.920961,-1.462235,1.860903,-1.720450,-0.746779,-0.989126,-1.134240,-1.149014,-0.690891,-0.987975,-0.973706,-0.700219
1,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-1.048979,-0.525509,-0.629966,-1.047442,-0.839852,-0.373498,-0.792891,0.208583,0.301344,-0.663030,-0.297599,-0.689624,-0.748999,-0.697633,0.427167,-0.065212,-0.738609,-0.655622,-0.383513,-0.708742,-0.849875,-0.147314,-0.974408,-0.363322,-0.944223,1.410778,-1.302693,-1.214305,-0.803414,-0.586684,-0.660136,-0.098134,-1.072261,-1.411103,-1.183576,-0.906749,-0.894424,-1.021086,-0.750317,-1.084885,-1.213390,-1.465964,-0.936581,-0.747907,-0.213146,-1.103646,-0.950324,-1.080212,-0.860915,-0.613991,-0.980197,-1.062303,-0.824470,-0.921642,-0.896293,-1.058423,-0.373498,-1.302535,-1.103646,-1.107154,-0.886914,-1.047932,1.758524,1.856723,1.825778,1.670201,1.909683,1.885189,1.602264,1.407766,0.785399,1.912801,-1.107797,-0.854711,1.920961,-1.131054,1.860903,-1.664183,-0.746779,-0.989126,-1.134240,-1.149014,-0.690891,-1.085868,-1.073406,-0.700219
2,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-0.993476,-0.480451,-0.629966,-0.991272,-0.839852,-1.385203,-1.002168,-0.771312,-0.435147,-0.663030,-1.566831,-1.533313,-0.748999,-0.697633,-0.088235,-1.299157,-0.311238,-0.655622,-0.813795,-0.342426,-0.849875,-1.213269,-0.784481,-1.348659,1.537735,-0.436789,-1.542000,-1.162914,-1.173704,-0.798247,-0.660136,-0.098134,-1.427526,-1.411103,-1.183576,-0.906749,-0.792204,-1.021086,-0.750317,-1.303366,-1.318110,-1.755803,-0.193775,-0.101828,-1.084675,-1.103646,-0.950324,-1.080212,-0.860915,-0.613991,-0.980197,-1.062303,-0.824470,-0.921642,-0.896293,-1.058423,-1.385203,-1.541932,-1.103646,-1.107154,-0.886914,-1.047932,1.758524,1.856723,1.825778,1.388775,1.712721,1.345790,1.465965,0.749915,0.611496,0.768106,-

In [20]:
df_processed.shape

(46, 91)

In [21]:
df_processed['co_curso'] = df['co_curso']
df_processed['no_curso'] = df['no_municipio'] + '_' + df['tp_grau_academico'] + '_' + df['no_curso']
df_processed['no_cine_area_geral'] = df['no_cine_area_geral']
df_processed['tp_grau_academico'] = df['tp_grau_academico']
df_processed['enade_faixa'] = df['enade_faixa']
df_processed['cpc_faixa'] = df['cpc_faixa']

In [22]:
df_processed.shape

(46, 97)

In [23]:
# df_processed = df_processed.drop(['qt_vg_total_diurno','qt_vg_total_noturno','qt_inscrito_total_diurno', 'qt_ing', 'qt_ing_0_17', 'qt_ing_25_29', 'qt_ing_30_34', 'qt_ing_40_49', 'qt_ing_50_59', 'qt_mat_30_34', 'qt_mat_60_mais', 'qt_mat_cornd', 'qt_conc_diurno', 'qt_conc_30_34', 'qt_conc_preta', 'qt_inscrito_total_noturno', 'qt_mat_0_17','qt_mat_35_39', 'qt_mat_amarela', 'qt_mat_indigena', 'qt_mat_40_49', 'qt_mat_50_59'],axis=1)

In [24]:
limit = 0.8
corr = df_processed.corr()
mask = np.triu(
    np.ones(corr.shape), k=1
).astype(bool)
corr_no_dial = corr.where(mask)
coll = [
    c
    for c in corr_no_dial.columns
    if any(abs(corr_no_dial[c])>limit)
]

C:\Users\ebony\AppData\Local\Temp\ipykernel_19856\2679276832.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df_processed.corr()


In [25]:
coll

['qt_inscrito_total_noturno',
 'qt_insc_vg_nova',
 'qt_ing_diurno',
 'qt_ing_noturno',
 'qt_ing_vg_nova',
 'qt_ing_enem',
 'qt_ing_vg_remanesc',
 'qt_ing_18_24',
 'qt_ing_35_39',
 'qt_ing_branca',
 'qt_ing_parda',
 'qt_mat',
 'qt_mat_fem',
 'qt_mat_masc',
 'qt_mat_diurno',
 'qt_mat_noturno',
 'qt_mat_0_17',
 'qt_mat_18_24',
 'qt_mat_25_29',
 'qt_mat_35_39',
 'qt_mat_40_49',
 'qt_mat_50_59',
 'qt_mat_branca',
 'qt_mat_preta',
 'qt_mat_parda',
 'qt_conc_fem',
 'qt_conc_masc',
 'qt_conc_noturno',
 'qt_conc_18_24',
 'qt_conc_25_29',
 'qt_conc_branca',
 'qt_conc_parda',
 'qt_ing_nacbras',
 'qt_mat_nacbras',
 'qt_conc_nacbras',
 'qt_aluno_deficiente',
 'qt_mat_deficiente',
 'qt_ing_rvredepublica',
 'qt_ing_rvetnico',
 'qt_ing_rvsocial_rf',
 'qt_ing_rvoutros',
 'qt_mat_reserva_vaga',
 'qt_mat_rvredepublica',
 'qt_mat_rvetnico',
 'qt_mat_rvsocial_rf',
 'qt_mat_rvoutros',
 'qt_ing_procescpublica',
 'qt_mat_procescpublica',
 'qt_mat_procescprivada',
 'qt_conc_procescprivada',
 'qt_mat_apoio_soci

In [26]:
df_processed = df_processed.drop(coll, axis=1)

In [27]:
df_processed['co_curso'] = df['co_curso']

In [28]:
df_processed.shape

(46, 44)

In [29]:
df_processed

,in_capital,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_remanesc,qt_inscrito_total,qt_inscrito_total_diurno,qt_insc_vg_remanesc,qt_ing,qt_ing_fem,qt_ing_masc,qt_ing_outra_forma,qt_ing_0_17,qt_ing_25_29,qt_ing_30_34,qt_ing_40_49,qt_ing_50_59,qt_ing_preta,qt_ing_amarela,qt_ing_cornd,qt_mat_30_34,qt_mat_60_mais,qt_mat_amarela,qt_mat_indigena,qt_mat_cornd,qt_conc,qt_conc_diurno,qt_conc_30_34,qt_conc_preta,qt_ing_deficiente,qt_ing_reserva_vaga,qt_sit_trancada,qt_sit_desvinculado,qt_ing_procescprivada,qt_conc_procescpublica,qt_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,co_curso,no_curso,no_cine_area_geral,tp_grau_academico,enade_faixa,cpc_faixa
0,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-0.860750,-0.372702,-0.839852,0.023957,-0.949849,0.780188,-0.697633,-0.088235,-1.593351,-0.998333,-0.708742,-0.849875,-0.784481,0.296756,-0.898681,-1.349439,-0.750317,-0.565178,-0.101828,-1.302557,-1.103646,-0.860915,-0.824470,-0.896293,1.103235,1.758524,-0.921816,-1.580111,-1.462235,-0.746779,-1.134240,-0.690891,-0.987975,1516690.0,belo_jardim_bacharelado_engenharia_de_computacao,"engenharia,_producao_e_construcao",bacharelado,SC,SC
1,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-1.048979,-0.525509,-0.839852,-0.373498,-0.792891,0.208583,-0.697633,0.427167,-0.738609,-0.655622,-0.708742,-0.849875,-0.974408,-0.944223,1.410778,-1.183576,-0.750317,-0.936581,-0.747907,-0.213146,-1.103646,-0.860915,-0.824470,-0.896293,-0.886914,1.758524,-1.107797,-0.854711,-1.131054,-0.746779,-1.134240,-0.690891,-1.085868,1516692.0,belo_jardim_bacharelado_engenharia_de_controle...,"engenharia,_producao_e_construcao",bacharelado,SC,SC
2,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-0.993476,-0.480451,-0.839852,-1.385203,-1.002168,-0.771312,-0.697633,-0.088235,-0.311238,-0.655622,-0.342426,-0.849875,-0.784481,1.537735,-0.436789,-1.183576,-0.750317,-0.193775,-0.101828,-1.084675,-1.103646,-0.860915,-0.824470,-0.896293,-0.886914,1.758524,-0.735836,-1.471301,-1.594707,-0.746779,-1.134240,-0.690891,-1.085868,1516689.0,belo_jardim_bacharelado_engenharia_hidrica,"engenharia,_producao_e_construcao",bacharelado,SC,SC
3,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-1.003128,-0.488287,-0.839852,-0.770954,-0.112742,-0.771312,-0.697633,1.457970,-0.738609,-0.998333,-1.075057,-0.849875,-1.164335,0.296756,0.025103,-1.349439,-0.750317,-0.565178,-0.747907,-0.866793,-1.103646,-0.860915,-0.824470,-0.896293,-0.886914,1.758524,-1.107797,-1.507571,-1.362881,-0.746779,-1.134240,-0.690891,-0.987975,1516693.0,belo_jardim_bacharelado_engenharia_quimica,"engenharia,_producao_e_construcao",bacharelado,SC,SC
4,-1.247219,1.326041,1.262757,-0.673697,-0.121268,0.176924,0.469697,0.036101,1.107927,0.724365,0.698530,1.809486,2.939750,1.825618,-0.998333,0.390206,0.267104,1.114789,0.296756,-0.436789,-0.520124,0.363057,0.549030,-0.101828,-1.084675,-0.283867,0.156386,-0.824470,-0.896293,-0.886914,1.758524,0.937988,0.197120,-0.700519,0.889023,1.584558,-0.270647,-0.890082,1270379.0,cabo_de_santo_agostinho_bacharelado_engenharia...,"engenharia,_producao_e_construcao",bacharelado,4,4
5,-1.247219,1.326041,1.262757,-0.673697,-0.121268,-0.081288,0.260077,-0.839852,1.577647,0.619726,1.310964,-0.697633,1.457970,0.757190,1.743358,0.390206,1.384082,0.734935,1.537735,2.565508,0.143327,-0.750317,0.549030,-0.747907,1.094148,-0.635201,-0.279600,0.461144,-0.071703,0.108160,1.758524,0.380047,-0.129311,-0.832991,0.343755,0.354626,0.569842,-0.106938,1270383.0,cabo_de_santo_agostinho_bacharelado_engenharia...,"engenharia,_producao_e_construcao",bacharelado,SC,SC
6,-1.247219,1.326041,1.262757,-0.673697,-0.121268,-0.373285,0.023030,-0.693860,1.396985,-0.845210,2.250029,-0.697633,1.457970,-0.311238,0.029801,0.023890,1.942572,-0.404627,0.296756,1.872670,-0.188399,1.476431,1.291836,-0.747907,0.440501,-0.986535,-0.715586,-0.824470,-0.896293,-0.886914,1.758524,0.566027,-0.310661,-1.296644,-0.201512,0.743025,-0.690891,-0.204831,1270380.0,cabo_de_santo_agostinho_bacharelado_engenharia...,"engenh

In [30]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [31]:
categoricas = ['no_cine_area_geral','tp_grau_academico', 'enade_faixa', 'cpc_faixa']

In [32]:
matriz_enc = enc.fit_transform(df_processed[categoricas])

c:\Users\ebony\Mestrado\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [33]:
df_enc = pd.DataFrame(matriz_enc, columns=enc.get_feature_names_out())

In [34]:
X = pd.concat([df_enc.reset_index(drop=True),df_processed.drop(columns=categoricas).reset_index(drop=True)],axis=1)

In [35]:
X

,"no_cine_area_geral_agricultura,_silvicultura,_pesca_e_veterinaria","no_cine_area_geral_ciencias_naturais,_matematica_e_estatistica","no_cine_area_geral_ciencias_sociais,_comunicacao_e_informacao",no_cine_area_geral_computacao_e_tecnologias_da_informacao_e_comunicacao_(tic),no_cine_area_geral_educacao,"no_cine_area_geral_engenharia,_producao_e_construcao","no_cine_area_geral_negocios,_administracao_e_direito",no_cine_area_geral_servicos,tp_grau_academico_bacharelado,tp_grau_academico_licenciatura,tp_grau_academico_tecnologico,enade_faixa_2,enade_faixa_3,enade_faixa_4,enade_faixa_5,enade_faixa_SC,cpc_faixa_3,cpc_faixa_4,cpc_faixa_5,cpc_faixa_SC,in_capital,qt_vg_total,qt_vg_total_diurno,qt_vg_total_noturno,qt_vg_remanesc,qt_inscrito_total,qt_inscrito_total_diurno,qt_insc_vg_remanesc,qt_ing,qt_ing_fem,qt_ing_masc,qt_ing_outra_forma,qt_ing_0_17,qt_ing_25_29,qt_ing_30_34,qt_ing_40_49,qt_ing_50_59,qt_ing_preta,qt_ing_amarela,qt_ing_cornd,qt_mat_30_34,qt_mat_60_mais,qt_mat_amarela,qt_mat_indigena,qt_mat_cornd,qt_conc,qt_conc_diurno,qt_conc_30_34,qt_conc_preta,qt_ing_deficiente,qt_ing_reserva_vaga,qt_sit_trancada,qt_sit_desvinculado,qt_ing_procescprivada,qt_conc_procescpublica,qt_apoio_social,qt_conc_apoio_social,qt_ativ_extracurricular,co_curso,no_curso
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-0.860750,-0.372702,-0.839852,0.023957,-0.949849,0.780188,-0.697633,-0.088235,-1.593351,-0.998333,-0.708742,-0.849875,-0.784481,0.296756,-0.898681,-1.349439,-0.750317,-0.565178,-0.101828,-1.302557,-1.103646,-0.860915,-0.824470,-0.896293,1.103235,1.758524,-0.921816,-1.580111,-1.462235,-0.746779,-1.134240,-0.690891,-0.987975,1516690.0,belo_jardim_bacharelado_engenharia_de_computacao
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-1.048979,-0.525509,-0.839852,-0.373498,-0.792891,0.208583,-0.697633,0.427167,-0.738609,-0.655622,-0.708742,-0.849875,-0.974408,-0.944223,1.410778,-1.183576,-0.750317,-0.936581,-0.747907,-0.213146,-1.103646,-0.860915,-0.824470,-0.896293,-0.886914,1.758524,-1.107797,-0.854711,-1.131054,-0.746779,-1.134240,-0.690891,-1.085868,1516692.0,belo_jardim_bacharelado_engenharia_de_controle...
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-0.993476,-0.480451,-0.839852,-1.385203,-1.002168,-0.771312,-0.697633,-0.088235,-0.311238,-0.655622,-0.342426,-0.849875,-0.784481,1.537735,-0.436789,-1.183576,-0.750317,-0.193775,-0.101828,-1.084675,-1.103646,-0.860915,-0.824470,-0.896293,-0.886914,1.758524,-0.735836,-1.471301,-1.594707,-0.746779,-1.134240,-0.690891,-1.085868,1516689.0,belo_jardim_bacharelado_engenharia_hidrica
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.247219,-1.198009,0.069192,-0.673697,-1.515848,-1.003128,-0.488287,-0.839852,-0.770954,-0.112742,-0.771312,-0.697633,1.457970,-0.738609,-0.998333,-1.075057,-0.849875,-1.164335,0.296756,0.025103,-1.349439,-0.750317,-0.565178,-0.747907,-0.866793,-1.103646,-0.860915,-0.824470,-0.896293,-0.886914,1.758524,-1.107797,-1.507571,-1.362881,-0.746779,-1.134240,-0.690891,-0.987975,1516693.0,belo_jardim_bacharelado_engenharia_quimica
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.247219,1.326041,1.262757,-0.673697,-0.121268,0.176924,0.469697,0.036101,1.107927,0.724365,0.698530,1.809486,2.939750,1.825618,-0.998333,0.390206,0.267104,1.114789,0.296756,-0.436789,-0.520124,0.363057,0.549030,-0.101828,-1.084675,-0.283867,0.156386,-0.824470,-0.896293,-0.886914,1.758524,0.937988,0.197120,-0.700519,0.889023,1.584558,-0.270647,-0.890082,1270379.0,cabo_de_santo_agostinho_bacharelado_engenharia...
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-1.247219,1.326041,1.262757,-0.673697,-0.121268,-0.081288,0.260077,-0.839852,1.577647,0.619726,1.

In [36]:
X.shape

(46, 60)

In [37]:
X = X.drop('in_capital', axis=1)

In [38]:
X.shape

(46, 59)

In [39]:
X = X.drop(['qt_vg_total_diurno','qt_vg_total_noturno','qt_inscrito_total_diurno', 'qt_ing', 'qt_ing_0_17', 'qt_ing_25_29', 'qt_ing_30_34', 'qt_ing_40_49', 'qt_ing_50_59', 'qt_mat_30_34', 'qt_mat_60_mais', 'qt_mat_cornd', 'qt_conc_diurno', 'qt_conc_30_34', 'qt_conc_preta', 'qt_mat_amarela', 'qt_mat_indigena',],axis=1)

In [40]:
X.columns

Index(['no_cine_area_geral_agricultura,_silvicultura,_pesca_e_veterinaria',
       'no_cine_area_geral_ciencias_naturais,_matematica_e_estatistica',
       'no_cine_area_geral_ciencias_sociais,_comunicacao_e_informacao',
       'no_cine_area_geral_computacao_e_tecnologias_da_informacao_e_comunicacao_(tic)',
       'no_cine_area_geral_educacao',
       'no_cine_area_geral_engenharia,_producao_e_construcao',
       'no_cine_area_geral_negocios,_administracao_e_direito',
       'no_cine_area_geral_servicos', 'tp_grau_academico_bacharelado',
       'tp_grau_academico_licenciatura', 'tp_grau_academico_tecnologico',
       'enade_faixa_2', 'enade_faixa_3', 'enade_faixa_4', 'enade_faixa_5',
       'enade_faixa_SC', 'cpc_faixa_3', 'cpc_faixa_4', 'cpc_faixa_5',
       'cpc_faixa_SC', 'qt_vg_total', 'qt_vg_remanesc', 'qt_inscrito_total',
       'qt_insc_vg_remanesc', 'qt_ing_fem', 'qt_ing_masc',
       'qt_ing_outra_forma', 'qt_ing_preta', 'qt_ing_amarela', 'qt_ing_cornd',
       'qt_conc', 'qt_

In [41]:
X.shape

(46, 42)

In [42]:
X.to_csv('cursos-ufrpe-pos-processamento.csv',sep=';',index=False)

In [39]:
# profile = ProfileReport(X, title='Relatório de Dados Mínimo', minimal=True)
# profile.to_file('cursos_ufrpe_4.html')